In [1]:
import spotipy
import pymongo
import requests

from enum import auto, StrEnum

from spotipy.oauth2 import SpotifyClientCredentials

# Make sure you have created a spotify_credentials.py file in the same directory as this notebook.
# the contents should be:
# CLIENT_ID = 'YOUR_CLIENT_ID'
# CLIENT_SECRET_ID = 'YOUR CLIENT_SECRET_ID'
from spotify_credentials import CLIENT_ID, CLIENT_SECRET_ID

In [36]:
# authenticate and initialise the Spotipy object
auth_manager = SpotifyClientCredentials(CLIENT_ID, CLIENT_SECRET_ID)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [3]:
# How to get a track id? From a sharing link:
# https://open.spotify.com/track/1KamjM1JNDyyOd6uOyZs17?si=f39f725471284f97
# https://open.spotify.com/track/{TRACK_ID}?si=f39f725471284f97

# In a similar way, you can access data about artists, albums, playlists, etc.

song = sp.track('1KamjM1JNDyyOd6uOyZs17')

In [ ]:
# you can preview the whole response json here, but it may be a bit long and hard to read
# song

In [4]:
mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")

In [5]:
db = mongo_client['musicDB']

In [6]:
# when developing a piece of Python software and having a limited, hardcoded list of options, it's good practice to use Enums.
# Enums store names and values which cannot be easily edited and would typically be imported from a separate file.
# They can also be coordinated between different systems.
# you can think of these as a class consisting primarily of class variables.
# NOTE: the ENUM options are objects, to access the value you need to run: AppRouteEnum.CREATE.value
# It may seem escessively complicated but is used widely as a suggestion this is a limited list of options not to be edited thoughtlessly.
# It can be also used as a placeholder for values not known at the time of development.
# This is also one of the few cases in Python where you don't need to keep alphabetical order by default.
class AppRouteEnum(StrEnum):
    
    CREATE = auto() # 'create'
    FIND_ONE = auto() # 'find_one', etc.
    FIND = auto()
    INSERT_ONE = auto()
    INSERT_MANY = auto()
    TEST = auto()
    UPDATE_ONE = auto()
    UPDATE_MANY = auto()
    DELETE_ONE = auto()
    DELETE_MANY = auto()

    # the @classmethod decorator means you don't need to create an object to use this method,
    # but you will be using other methods or variables from this class.
    # when using need, you need to pass "cls" as the first argument during class definition.
    @classmethod
    def get_names(cls):
        return [member.name for member in cls]

    @classmethod
    def get_values(cls):
        return [member.value for member in cls]

    @classmethod
    def to_dict(cls):
        return {member.name: member.value for member in cls}

In [7]:
# you can iterate over the enum, but you can see that
list(AppRouteEnum)

[<AppRouteEnum.CREATE: 'create'>,
 <AppRouteEnum.FIND_ONE: 'find_one'>,
 <AppRouteEnum.FIND: 'find'>,
 <AppRouteEnum.INSERT_ONE: 'insert_one'>,
 <AppRouteEnum.INSERT_MANY: 'insert_many'>,
 <AppRouteEnum.TEST: 'test'>,
 <AppRouteEnum.UPDATE_ONE: 'update_one'>,
 <AppRouteEnum.UPDATE_MANY: 'update_many'>,
 <AppRouteEnum.DELETE_ONE: 'delete_one'>,
 <AppRouteEnum.DELETE_MANY: 'delete_many'>]

In [8]:
# you can't access these through indexing
AppRouteEnum[0]

KeyError: 0

In [9]:
# but if cast to a list, you can see it's an object
enum_member = list(AppRouteEnum)[0]
print(enum_member, type(enum_member))

create <enum 'AppRouteEnum'>


In [ ]:
AppRouteEnum.to_dict()

In [ ]:
AppRouteEnum.get_names()

In [10]:
# If we want a single value from an Enum, this is how we access it:
# Again, may look like an overkill, but this is to make sure these things are not edited by accident and kept consistent
# It also allows to avoid typos when you need to use the same string value over and over again
AppRouteEnum.CREATE.value

'create'

In [ ]:
AppRouteEnum.DELETE_ONE.value

In [ ]:
# Let's get back to the database! And the "favSongs" collection

In [11]:
# combine all the info into one dict that we will want to save to the db
def get_track_info(track_info):
    # track_info = sp.track(track_id)

    artist_names = [artist['name'] for artist in track_info['artists']]
    release_year = track_info['album']['release_date'].split('-')[0]
    spotify_url = track_info['external_urls']['spotify']

    track_data = {
        'artists': artist_names,
        'spotify_id': track_info['id'],
        'name': track_info['name'],
        'release_year': release_year,
        'spotify_url': spotify_url,
    }
    return track_data

In [14]:
track_id = '1KamjM1JNDyyOd6uOyZs17'
track_data = sp.track(track_id) # raw data from Spotify API
track_info = get_track_info(track_data)

In [15]:
track_info

{'artists': ['Solomun', 'Isolation Berlin'],
 'spotify_id': '1KamjM1JNDyyOd6uOyZs17',
 'name': 'Kreatur der Nacht (feat. Isolation Berlin)',
 'release_year': '2020',
 'spotify_url': 'https://open.spotify.com/track/1KamjM1JNDyyOd6uOyZs17'}

In [16]:
collection = db['favSongs']

In [17]:
collection.find_one()

In [20]:
collection.count_documents({})

0

In [21]:
FLASK_URL = "http://localhost:5000"

def get_url(method_route, app_url=FLASK_URL):
    return '/'.join([app_url, method_route])

In [22]:
# let's preview the test URL
test_route = AppRouteEnum.TEST.value
print(f'test route: {test_route}')

TEST_URL = get_url(test_route)
print('TEST URL', TEST_URL)

test route: test
TEST URL http://localhost:5000/test


In [23]:
# test the Flask app and database connection
resp = requests.get(TEST_URL)
resp.json()

{'collections': ['favSongs', 'SongInfo'], 'success': True}

In [24]:
# using this query, we'll be filetring documents int the database
query = {'spotify_id': track_info['spotify_id']}

In [25]:
# find all documents matching the query
resp_find = requests.get(
    str(get_url(AppRouteEnum.FIND.value)), # the get_url function returns a str but for some reason you need to cast the output to str here or export it to a variable
    json={'query': query},
)
resp_find.json()

[]

In [26]:
resp_find

<Response [200]>

In [27]:
get_url(AppRouteEnum.INSERT_ONE.value)

'http://localhost:5000/insert_one'

In [28]:
# Now, let's try to add the same document again, using the POST request
resp_insert_one = requests.post(
    str(get_url(AppRouteEnum.INSERT_ONE.value)),
    json={'data': track_info},
)

In [29]:
resp_insert_one

<Response [200]>

In [30]:
print(resp_insert_one.json())

{'success': True}


In [31]:
# How many documents matching the query now?
resp_find = requests.get(
    str(get_url(AppRouteEnum.FIND.value)),
    json={'query': query},
)
# How many documents match the query? (please keep ini mind this may be slow with a large database)
len(resp_find.json())

1

In [32]:
resp_del_many = requests.delete(
    str(get_url(AppRouteEnum.DELETE_MANY.value)),
    json={'query': query},
)

In [33]:
resp_del_many.text

'{\n  "success": true\n}\n'

In [34]:
# How many now?
resp_find = requests.get(
    str(get_url(AppRouteEnum.FIND.value)),
    json={'query': query},
)
# How many documents match the query? (please keep ini mind this may be slow with a large database)
len(resp_find.json())

0

In [ ]:
# Now, let's try to add a whole playlist to the database!
# Again we'll be working with code from week09/class_demo.ipynb

In [37]:
# from week09/class_demo.ipynb
# playlist url
# https://open.spotify.com/playlist/37i9dQZF1DX7iB3RCnBnN4?si=9d199f5212064db7
PLAYLIST_ID = '37i9dQZF1DX7iB3RCnBnN4'
playlist_info = sp.playlist(PLAYLIST_ID)

In [38]:
# get the tracks info from the playlist data

tracks = playlist_info['tracks']

In [39]:
# spotipy returns tracks info on pages, max 100 results at once
# You can iterate over "pages", using the "next" key
# reference:
# https://stackoverflow.com/questions/39086287/spotipy-how-to-read-more-than-100-tracks-from-a-playlist
playlist_data = []

while tracks['next']:
    tracks = sp.next(tracks)
    playlist_data.extend(tracks['items'])

In [40]:
len(playlist_data)

531

In [41]:
playlist_info = [
    get_track_info(track_data['track'])
    for track_data in playlist_data
]

In [42]:
len(playlist_info)

531

In [43]:
# Let's see the last entry
playlist_info[-1]

{'artists': ['Adele'],
 'spotify_id': '4sPmO7WMQUAf45kwMOtONw',
 'name': 'Hello',
 'release_year': '2016',
 'spotify_url': 'https://open.spotify.com/track/4sPmO7WMQUAf45kwMOtONw'}

In [44]:
# Now, let's try to add the same document again, using the POST request
resp_insert_many = requests.post(
    str(get_url(AppRouteEnum.INSERT_MANY.value)),
    json={'data': playlist_info},
)

In [45]:
# How many documents matching the query now?
resp_find = requests.get(
    str(get_url(AppRouteEnum.FIND.value)),
    json={'query': {}},
)
# How many documents match the query? (please keep ini mind this may be slow with a large database)
len(resp_find.json())

531

In [1]:
# Now, we're going back to pymongo for a bit, without the Flask API.

In [ ]:
resp_insert_one

In [46]:
db.favSongs.count_documents({})

531

In [47]:
db.favSongs.count_documents({'release_year': '2000'})

2

In [48]:
# Let's see the oldest song
db.favSongs.find_one(sort=[('release_year', pymongo.ASCENDING)])

{'_id': ObjectId('663d25770101c5c3e4f3c3f4'),
 'artists': ['Bing Crosby',
  'Ken Darby Singers',
  'John Scott Trotter & His Orchestra'],
 'spotify_id': '4so0Wek9Ig1p6CRCHuINwW',
 'name': 'White Christmas - 1947 Version',
 'release_year': '1942',
 'spotify_url': 'https://open.spotify.com/track/4so0Wek9Ig1p6CRCHuINwW'}

In [49]:
# Let's see the most recent song
db.favSongs.find_one(sort=[('release_year', pymongo.DESCENDING)])

{'_id': ObjectId('663d25770101c5c3e4f3c2b8'),
 'artists': ['Jung Kook', 'Latto'],
 'spotify_id': '7x9aauaA9cu6tyfpHnqDLo',
 'name': 'Seven (feat. Latto)',
 'release_year': '2023',
 'spotify_url': 'https://open.spotify.com/track/7x9aauaA9cu6tyfpHnqDLo'}

In [50]:
# Let's find all songs by Rihanna
cursor = db.favSongs.find({'artists': 'Rihanna'})
cursor[0]

{'_id': ObjectId('663d25770101c5c3e4f3c25b'),
 'artists': ['Eminem', 'Rihanna'],
 'spotify_id': '15JINEqzVMv3SvJTAXAKED',
 'name': 'Love The Way You Lie',
 'release_year': '2010',
 'spotify_url': 'https://open.spotify.com/track/15JINEqzVMv3SvJTAXAKED'}

In [51]:
cursor_count = len(list(cursor))
cursor_count

13

In [ ]:
# delete all data from the collection
# db.favSongs.delete_many({})

In [ ]:
# Exercise:
# Can you add the "find minimum" / "find maximum" functionality
# to the Flask app and interact with it through the API?